<h1 style="text-align:center">Seafood Business Analysis with Yelp Data </h1>
<h4 style="text-align:right">Kangyi Zhao, Xinyu Zhang, Kehui Yao </h4>

# 1 Introduction

## 1.1 Thesis Statement

goal motivation method(check Nightlife in Las Vegas report.pdf for insights)

## 1.2 Data Background

(check data description document for insights)

# 2 Data Filtering

talk about how to extract seafood related observations from business.json and review.json

include seafood restaurant not include steakhouse 

the length because we only want restaurant focusing on seafood and don't want other thing to interfere

the number of review > 50 because we don't want some outlier reviews

we save our new data set as ".csv".

# 3 Attribute Analysis

## 3.1 

# 4 Review Analysis

## 4.1 Data Cleaning

First, we obtain review data set with features we need. The review data from "all_review.csv" contains 7 features. We only keep features: 'business_id', 'stars' and 'text'. Then we save the new review data set as "review_with_useful_features.csv".

Second, we do **word tokenization**, which means we need to convert text into words. This process has 7 steps: step1, convert "n't" to "not" and then connect "not" with the word after it, such as changing "wouldn't go" to "would not_go"; step2, break paragraph into words; step3, remove punctuation, nonalphbetic string; step4, convert numbers to words; step5, convert words to lower case; step6, remove stopwords (we import "stopwords.words('english')" from python package nltk.corpus); step7, do lemmatization, such as changing "likes" to "like".

## 4.2 Sentiment Analysis

### 4.2.1 positive and negative adjectives classification

We use **Multinomial Naive Bayes Classifier** to classify adjectives in review text as positive and negative.

First, we define stars from 1 to 3 as negative and stars from 4 to 5 as positive. Then we convert stars into positive/negative tags and treat positive/negative tag as response variable. Second, we extract adjectives from tokenized words by python function "nltk.pos_tag()" (part-of-speech tagging). Third, we count frequency of each adjectives in all review texts and obtain 1200 most frequent adjectives. Fourth, we count occurrences for these 1200 adjectives in each review text and obtain the frequency matrix with the index of review as row and 1200 adjectives as column. We treat this frequency matrix as design matrix. Fifth, we fit multinomial Naive Bayes model with design matrix and response variable. Sixth, we do pridiction with each adjective and the positive/negative prediction result is the sentiment tag for adjectives.

We save the dictionary for adjectives with positive/negative tags as "dict_adj.txt".

### 4.2.2 informative nouns in reviews

First, we extract nouns from tokenized words by python function "nltk.pos_tag()" (part-of-speech tagging). Second, we count frequency of each nouns in all review texts. Because we assume that informative nouns would show up many times in review text, we only consider nouns with frequency larger than 4000. Third, we pick up informative nouns from them manually. 

**Informative nouns** are: food, lobster, crab, shrimp, oyster, fish, clam, service, waiter, waitress, chef, manager, price.

### 4.2.3 sentiment analysis for informative nouns

From this part, we do counting at the restaurant level instead of review level. That is to say, we would count some kind of word from all the review texts of each restaurant respectively.

Also, we need to obtain stars for each restaurant from "seafood_business.csv".

Then we obtain **sentiment table** for each informative nouns. First, we count the number of positive/negative adjectives near each informative noun. Second, we compute the proportion of positive adjectives among all the nearby adjectives. The **sentiment table for food** is an example shown below.

(插food table图)

## 4.3 Test and Finding

First, we do **ANOVA** and **F-test** to see whether there is any difference between informative nouns.

to be continue...

ANOVA between food/service/price

ANOVA inside food

ANOVA inside service

Second, we compute **correlation between the restaurant stars and the proportion of positive adjectives**.

barplot for food/service/price

barplot inside food

barplot inside service


# 5 Conclusion and Suggestion

## 5.1 conclusion

## 5.2 suggestion


# 6 Strength and Weakness

## 6.1 Strength

## 6.2 Weakness

# 7 Contribution

# 8 Reference